In [53]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
def read_data(filename):
    return pd.read_csv(filename)

In [3]:
def split_data(df, x, y):
    X = df[x]
    y = df[y]
    return X, y

In [4]:
def remove_stop_words_and_lower(x):
    list_stopwords = stopwords.words('english')
    
    temp_x = x.copy()
    new_x = []
    
    for i in temp_x:
        words = i.split()
        sentence = []
        for j in words:
            if j not in list_stopwords:
                sentence.append(j.lower())
        new_x.append(' '.join(sentence))
        
    return new_x

In [5]:
def remove_punctuation(x):
    temp_x = x.copy()
    new_x = []
    
    for i in temp_x:
        for j in string.punctuation:
            i = i.replace(j, '')
        new_x.append(i)
        
    return new_x

In [29]:
class Token_Data:
    def __init__(self):
        self.tokenizer = TfidfVectorizer(max_df=0.9, min_df=5)
    def fit(self, data, y=None):
        self.tokenizer.fit(data)
    def transform(self, data, y=None):
        return self.tokenizer.transform(data)
    def fit_transform(self, data, y=None):
        self.fit(data)
        return self.transform(data)

In [30]:
# READ CSV DATA
df = read_data('train.csv')

In [31]:
# SPLIT X AND Y
X, y = split_data(df, 'text', 'target')

X = X.to_list()
y = y.to_list()

In [39]:
# TRAIN TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [48]:
# CLEAN DATA FOR X_TRAIN
x = remove_stop_words_and_lower(X_train)
x = remove_punctuation(x)
tokenizer = Token_Data()
x = tokenizer.fit_transform(x)

In [51]:
# CREATE CLASSIFIER
random_forest = RandomForestClassifier()
random_forest.fit(x, y_train)

RandomForestClassifier()

In [52]:
# CLEAN DATA FOR X_TEST
x = remove_stop_words_and_lower(X_test)
x = remove_punctuation(x)
x = tokenizer.transform(x)

In [54]:
# TEST PREDICTION
y_pred = random_forest.predict(x)
accuracy_score(y_test, y_pred)

0.7715036112934996

In [59]:
# READ CSV DATA
df1 = read_data('test.csv')

# CLEAN DATA FOR SUBMISSION
submission = df1['text']
x = remove_stop_words_and_lower(submission)
x = remove_punctuation(x)
x = tokenizer.transform(x)

In [61]:
# FINAL PREDICTION
y_pred = random_forest.predict(x)

In [69]:
# CREATE SUBMISSION FILE
id = df1['id'].to_list()
target = y_pred.tolist()

# CREATE DICTIONARY FOR DATAFRAME
dataframe_dictionary = {
    'id': id,
    'target': target
}

# CREATE DATAFRAME
submission_df = pd.DataFrame(dataframe_dictionary, index=None)
submission_df.to_csv('submission.csv',)